# Seagrass Area Calculation

Calculate ground truth vs predicted seagrass area per ortho using the test split.
Saves results to `outputs/eval-area/` for visualization in `eval-area-plot.ipynb`.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import re
import torch
import numpy as np
import pandas as pd
import albumentations as A
from tqdm.auto import tqdm

from src.models.smp import SMPMulticlassSegmentationModel

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

# Create output directory
OUTPUT_DIR = Path("../outputs/eval-area")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

Device: cuda


In [2]:
def extract_ortho_name(filename: str) -> str:
    """Extract ortho name from chip filename.
    
    Examples:
    - koeye_u0388_42.npz -> koeye_u0388
    - pruth_bay_u0699_123.npz -> pruth_bay_u0699
    """
    stem = Path(filename).stem
    match = re.match(r"(.+)_(\d+)$", stem)
    if match:
        return match.group(1)
    return stem


def count_class_pixels(mask: np.ndarray, class_id: int = 1, ignore_index: int = -100) -> int:
    """Count pixels belonging to a class, excluding ignored pixels."""
    valid_mask = mask != ignore_index
    return int(np.sum((mask == class_id) & valid_mask))

In [3]:
# Paths
METADATA_PATH = Path("../metadata_subset.csv")
TEST_CHIP_DIR = Path("/mnt/class_data/sdalgarno/prototype_frac_75/chips_1024/test")
CKPT_PATH = Path("/mnt/class_data/sdalgarno/checkpoints/segformer-train50/segformer_train50_epoch-299_val-iou-0.8837.ckpt")

# Load metadata and create lookup dicts
metadata_df = pd.read_csv(METADATA_PATH)
ortho_to_resolution = dict(zip(metadata_df["site"], metadata_df["resolution_cm"]))
ortho_to_region = dict(zip(metadata_df["site"], metadata_df["region"]))

print(f"Loaded metadata for {len(metadata_df)} orthos")
print(f"Regions: {metadata_df['region'].unique()}")

# Load test chips
test_chips = sorted(TEST_CHIP_DIR.glob("*.npz"))
print(f"Found {len(test_chips)} test chips")

# Test transforms (same as config)
test_transforms = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
    ),
    A.ToTensorV2(),
])

# Load model
model = SMPMulticlassSegmentationModel.load_from_checkpoint(CKPT_PATH, map_location=DEVICE)
model.eval()
model.to(DEVICE)
print("Model loaded")

Loaded metadata for 67 orthos
Regions: ['South' 'North' 'Central']
Found 1761 test chips
Model loaded


In [4]:
# Per-chip area calculation
results = []

with torch.no_grad():
    for chip_path in tqdm(test_chips, desc="Processing chips"):
        # Extract ortho name and get metadata
        ortho = extract_ortho_name(chip_path.name)
        resolution_cm = ortho_to_resolution.get(ortho)
        region = ortho_to_region.get(ortho)
        
        if resolution_cm is None:
            print(f"Warning: No metadata for ortho '{ortho}' (chip: {chip_path.name})")
            continue
        
        # Load chip
        data = np.load(chip_path)
        image = data["image"]
        label = data["label"]
        
        # Apply transforms and run inference
        augmented = test_transforms(image=image, mask=label)
        image_tensor = augmented["image"].unsqueeze(0).to(DEVICE)
        
        logits = model(image_tensor)
        pred = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy()
        
        # Count seagrass pixels
        gt_pixels = count_class_pixels(label, class_id=1, ignore_index=-100)
        pred_pixels = count_class_pixels(pred, class_id=1, ignore_index=-100)
        
        # Calculate areas
        pixel_area_m2 = (resolution_cm / 100) ** 2
        gt_area_m2 = gt_pixels * pixel_area_m2
        pred_area_m2 = pred_pixels * pixel_area_m2
        
        results.append({
            "chip_filename": chip_path.name,
            "ortho": ortho,
            "region": region,
            "resolution_cm": resolution_cm,
            "gt_pixels": gt_pixels,
            "pred_pixels": pred_pixels,
            "gt_area_m2": gt_area_m2,
            "pred_area_m2": pred_area_m2,
        })

chip_df = pd.DataFrame(results)
print(f"Processed {len(chip_df)} chips")
chip_df.head()

Processing chips:   0%|          | 0/1761 [00:00<?, ?it/s]

Processed 1761 chips


,chip_filename,ortho,region,resolution_cm,gt_pixels,pred_pixels,gt_area_m2,pred_area_m2
0,bag_harbour_u0490_100.npz,bag_harbour_u0490,North,2.7,100805,207327,73.486845,151.141383
1,bag_harbour_u0490_101.npz,bag_harbour_u0490,North,2.7,0,9957,0.000000,7.258653
2,bag_harbour_u0490_1015.npz,bag_harbour_u0490,North,2.7,0,0,0.000000,0.000000
3,bag_harbour_u0490_1016.npz,bag_harbour_u0490,North,2.7,165667,162829,120.771243,118.702341
4,bag_harbour_u0490_1017.npz,bag_harbour_u0490,North,2.7,762942,701010,556.184718,511.036290


In [5]:
# Aggregate by ortho
ortho_df = chip_df.groupby(["ortho", "region"]).agg({
    "gt_pixels": "sum",
    "pred_pixels": "sum",
    "resolution_cm": "first",
}).reset_index()

# Recalculate areas from aggregated pixels
ortho_df["gt_area_m2"] = ortho_df["gt_pixels"] * (ortho_df["resolution_cm"] / 100) ** 2
ortho_df["pred_area_m2"] = ortho_df["pred_pixels"] * (ortho_df["resolution_cm"] / 100) ** 2
ortho_df["area_error_m2"] = ortho_df["pred_area_m2"] - ortho_df["gt_area_m2"]
ortho_df["abs_error_m2"] = ortho_df["area_error_m2"].abs()

print(f"Aggregated to {len(ortho_df)} orthos")
ortho_df.head(10)

Aggregated to 12 orthos


,ortho,region,gt_pixels,pred_pixels,resolution_cm,gt_area_m2,pred_area_m2,area_error_m2,abs_error_m2
0,bag_harbour_u0490,North,70556265,72930288,2.7,51435.517185,53166.179952,1730.662767,1730.662767
1,beck_u0409,South,16048845,12696795,5.0,40122.112500,31741.987500,-8380.125000,8380.125000
2,mcmullin_north_u0900,Central,25361335,27486982,2.3,13416.146215,14540.613478,1124.467263,1124.467263
3,mcmullin_north_u1270,Central,22557954,19776298,2.5,14098.721250,12360.186250,-1738.535000,1738.535000
4,section_cove_u0249,North,5697545,5098348,4.4,11030.447120,9870.401728,-1160.045392,1160.045392
5,section_cove_u0487,North,6534395,4353941,4.0,10455.032000,6966.305600,-3488.726400,3488.726400
6,sedgwick_u0085,North,4679398,4291081,4.4,9059.314528,8307.532816,-751.781712,751.781712
7,sedgwick_u0260,North,5526793,3490058,4.1,9290.539033,5866.787498,-3423.751535,3423.751535
8,sedgwick_u0482,North,5713514,4342004,4.0,9141.622400,6947.206400,-2194.416000,2194.416000
9,triquet_bay_u0537,Central,20614667,19411806,4.0,32983.467200,31058.889600,-1924.577600,1924.577600


In [6]:
# Save results
chip_df.to_csv(OUTPUT_DIR / "chip_areas.csv", index=False)
ortho_df.to_csv(OUTPUT_DIR / "ortho_areas.csv", index=False)

print(f"Saved chip results to: {OUTPUT_DIR / 'chip_areas.csv'}")
print(f"Saved ortho results to: {OUTPUT_DIR / 'ortho_areas.csv'}")

# Summary stats
print(f"\nSummary:")
print(f"  Total GT area: {ortho_df['gt_area_m2'].sum():,.1f} m²")
print(f"  Total Pred area: {ortho_df['pred_area_m2'].sum():,.1f} m²")
print(f"  Overall MAE: {ortho_df['abs_error_m2'].mean():,.1f} m²")

Saved chip results to: ../outputs/eval-area/chip_areas.csv
Saved ortho results to: ../outputs/eval-area/ortho_areas.csv

Summary:
  Total GT area: 270,957.0 m²
  Total Pred area: 242,161.8 m²
  Overall MAE: 2,875.5 m²
